In [ ]:
# supermarket_dashboard.py 
# --------------------------------------------------------
# Requires: dash, dash-bootstrap-components, plotly, pandas, numpy, statsmodels

import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc, dash_table, Input, Output
import plotly.graph_objects as go
from datetime import datetime
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX

# ========================================================
# 1) SYNTHETIC MASTER DATA  
# ========================================================
TODAY = pd.to_datetime("2025-01-15")
np.random.seed(1)

PRODUCT_NAMES  = ["OUR_BRAND"] + [f"COMP{i}_BRAND" for i in range(1, 10)]
DATES_SALES = pd.date_range("2023-01-31", periods=24, freq="M")

sales_dict = {
    prod: pd.Series(np.random.randint(50, 300, len(DATES_SALES)), index=DATES_SALES)
    for prod in PRODUCT_NAMES
}

phaseout_df = pd.DataFrame({
    "Product": PRODUCT_NAMES[1:],
    "PhaseOutDate": [
        (TODAY + relativedelta(months=int(np.random.randint(3, 18)))).date()
        for _ in PRODUCT_NAMES[1:]
    ]
})
phaseout_df["LikelyPhaseOut"] = phaseout_df["PhaseOutDate"].apply(
    lambda d: "YES" if relativedelta(pd.to_datetime(d), TODAY).months <= 6 else "NO"
)

retailers = [f"Retailer_{i}" for i in range(30)]
quarters  = pd.date_range("2024-03-31", periods=4, freq="Q")
targets = pd.DataFrame({
    "Retailer": retailers,
    "FirstOrder": np.random.choice(quarters, len(retailers)),
    "SalesPotential": np.random.randint(30, 300, len(retailers)),
    "VisitsYTD": np.random.randint(1, 25, len(retailers)),
    "SalesCaptured": np.random.randint(0, 150, len(retailers)),
    "CompBrand": np.random.choice(phaseout_df["Product"], len(retailers))
}).merge(phaseout_df, left_on="CompBrand", right_on="Product", how="left")

def segment_fn(row):
    if row["FirstOrder"] <= quarters[1]:
        return "Early Adopter"
    if row["LikelyPhaseOut"] == "YES":
        return "Near-Term Opportunity"
    return "Long-Term Prospect"

targets["Segment"] = targets.apply(segment_fn, axis=1)
targets["SalesRatio"] = (targets["SalesCaptured"] / targets["SalesPotential"]).round(2)
targets["VisitEfficiency"] = (targets["SalesCaptured"] / targets["VisitsYTD"]).round(2)

adopt_dates  = pd.date_range("2023-06-30", periods=6, freq="Q")
adopt_values = [5, 12, 18, 25, 35, 47]

# ========================================================
# 2) HELPER FUNCTIONS
# ========================================================
def sarimax_forecast(series, steps=6):
    model  = SARIMAX(series, order=(1, 0, 1),
                     seasonal_order=(1, 0, 0, 12),
                     enforce_stationarity=False,
                     enforce_invertibility=False)
    res    = model.fit(disp=False)
    idx_fc = pd.date_range(series.index[-1] + relativedelta(months=1), periods=steps, freq="M")
    fc_res = res.get_forecast(steps=steps)
    return idx_fc, fc_res

def build_sales_fig(prod):
    series = sales_dict[prod]
    idx_fc, fc = sarimax_forecast(series)
    ci = fc.conf_int(alpha=0.2); ci.index = idx_fc

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=series.index, y=series.values,
                             mode="lines+markers", name="Historical"))
    fig.add_trace(go.Scatter(x=idx_fc, y=fc.predicted_mean,
                             mode="lines+markers", line=dict(dash="dash"),
                             name="Forecast"))
    fig.add_trace(go.Scatter(x=idx_fc, y=ci.iloc[:, 0], mode="lines",
                             line=dict(width=0), showlegend=False))
    fig.add_trace(go.Scatter(x=idx_fc, y=ci.iloc[:, 1], mode="lines",
                             line=dict(width=0), fill="tonexty",
                             fillcolor="rgba(0,100,200,0.15)", name="80% CI"))
    fig.update_layout(height=320, title=f"{prod} – 6-Month Sales Forecast",
                      margin=dict(l=30, r=15, t=40, b=30))
    return fig

def capture_bar(seg):
    grp = targets[targets["Segment"] == seg].groupby("Segment")["SalesRatio"].mean()
    if grp.empty:
        grp = pd.Series([0], index=[seg])
    fig = go.Figure(go.Bar(x=grp.index, y=grp.values,
                           text=grp.round(2), textposition="auto"))
    fig.update_layout(title=f"Avg Sales Ratio – {seg}", height=220,
                      margin=dict(l=30, r=15, t=35, b=30), yaxis=dict(range=[0, 1]))
    return fig

def scatter_activity_capture(seg):
    d = targets[targets["Segment"] == seg]
    fig = go.Figure(go.Scatter(x=d["VisitsYTD"], y=d["SalesRatio"], mode="markers",
                               marker=dict(size=d["SalesPotential"] / 20,
                                           color=d["SalesRatio"], colorscale="Blues",
                                           showscale=True),
                               text=d["Retailer"]))
    fig.update_layout(title=f"Visits vs Sales Ratio – {seg}", height=300,
                      xaxis_title="Visits YTD", yaxis_title="Sales Ratio",
                      margin=dict(l=30, r=15, t=40, b=35))
    return fig

def ytd_bar_fig():
    ytd = {prod: int(series.sum()) for prod, series in sales_dict.items()}
    fig = go.Figure(go.Bar(x=list(ytd.keys()), y=list(ytd.values()),
                           text=list(ytd.values()), textposition="auto"))
    fig.update_layout(title="YTD Sales – Our Brand vs Competitors",
                      height=280, margin=dict(l=30, r=15, t=40, b=30))
    return fig

def segment_kpi_table():
    seg_kpi_df = (targets.groupby("Segment")
                  .agg(SalesPotential=("SalesPotential", "sum"),
                       SalesCaptured=("SalesCaptured", "sum"),
                       Visits=("VisitsYTD", "sum"))
                  .assign(SalesRatio=lambda d: d["SalesCaptured"] / d["SalesPotential"])
                  .reset_index())
    return dash_table.DataTable(
        data=seg_kpi_df.round(1).to_dict("records"),
        columns=[{"name": c, "id": c} for c in seg_kpi_df.columns],
        style_cell={"fontSize": 12, "padding": "4px"},
        style_header={"backgroundColor": "#eef", "fontWeight": "bold"},
        page_size=4
    )

def insights(seg, retail, brand):
    out = []
    avg_cap = targets[targets["Segment"] == seg]["SalesRatio"].mean()
    if avg_cap < 0.25:
        out.append(f"⚠️ Low conversion ({avg_cap:.1%}) in {seg}; review strategy.")
    if retail:
        row = targets[targets["Retailer"] == retail].iloc[0]
        if row["SalesRatio"] < 0.2 and row["VisitsYTD"] > 10:
            out.append("🔍 High visits but low conversion; investigate fit.")
        elif row["SalesRatio"] > 0.6:
            out.append("✅ Strong traction; highlight success.")
    if brand != "OUR_BRAND":
        row = phaseout_df[phaseout_df["Product"] == brand].iloc[0]
        months_left = relativedelta(pd.to_datetime(row["PhaseOutDate"]), TODAY).months
        if months_left <= 6:
            out.append(f"🔔 {brand} phasing out in {months_left} months; plan shelf replacement.")
        idx_fc, fc = sarimax_forecast(sales_dict[brand])
        if (fc.predicted_mean < 0).any():
            months = ", ".join(idx_fc[fc.predicted_mean < 0].strftime("%b-%Y"))
            out.append(f"🔹 Sales decline expected in {months}. Trigger promo campaigns.")
    if not out:
        out.append("ℹ️ No special insights for this selection.")
    return out

# ========================================================
# 3) DASH APP & LAYOUT
# ========================================================
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

seg_opts = [{"label": s, "value": s} for s in sorted(targets["Segment"].unique())]
retail_opts = lambda s: [{"label": i, "value": i} for i in sorted(targets[targets["Segment"] == s]["Retailer"].unique())]
brand_opts = lambda i: [{"label": "OUR_BRAND", "value": "OUR_BRAND"}] + [{
    "label": targets.loc[targets["Retailer"] == i, "CompBrand"].iloc[0],
    "value": targets.loc[targets["Retailer"] == i, "CompBrand"].iloc[0]
}]

adopt_fig = go.Figure(go.Scatter(x=adopt_dates, y=adopt_values, mode="lines+markers"))
adopt_fig.update_layout(height=220, title="Cumulative Customer Adoption – OUR_BRAND",
                        margin=dict(l=30, r=15, t=40, b=30))

app.layout = dbc.Container(fluid=True, children=[
    html.H4("OUR_BRAND — Sales & Retail Strategy Dashboard", className="mt-3 mb-4"),

    dbc.Row([
        dbc.Col(dcc.Dropdown(id="seg-dd", options=seg_opts,
                             value=seg_opts[0]["value"], clearable=False), md=4),
        dbc.Col(dcc.Dropdown(id="retail-dd", clearable=False), md=4),
        dbc.Col(dcc.Dropdown(id="brand-dd", clearable=False), md=4),
    ], className="mb-3"),

    dbc.Row([
        dbc.Col(dcc.Graph(id="sales-fig"), md=6),
        dbc.Col(dcc.Graph(figure=adopt_fig, config={"displayModeBar": False}), md=6),
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id="cap-bar"), md=6),
        dbc.Col(dcc.Graph(id="scatter-ac"), md=6),
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dbc.Card([dbc.CardHeader("Segment KPI Snapshot"),
                          dbc.CardBody(segment_kpi_table())]), md=6),
        dbc.Col(dbc.Card([dbc.CardHeader("Insights"),
                          dbc.CardBody(html.Ul(id="insight-ul", style={"fontSize": "0.9rem"}))]), md=6)
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dcc.Graph(figure=ytd_bar_fig(), config={"displayModeBar": False}), md=12)
    ], className="mb-4"),

    dash_table.DataTable(
        data=targets.to_dict("records"),
        columns=[{"name": c, "id": c} for c in
                 ["Retailer", "Segment", "SalesPotential", "SalesCaptured",
                  "SalesRatio", "VisitsYTD", "CompBrand", "PhaseOutDate"]],
        page_size=12,
        sort_action="native",
        style_cell={"fontSize": 12, "padding": "4px", "whiteSpace": "nowrap"},
        style_header={"backgroundColor": "#f0f0f0", "fontWeight": "bold"},
        id="tbl"
    )
])

@app.callback(
    Output("retail-dd", "options"), Output("retail-dd", "value"),
    Input("seg-dd", "value")
)
def update_retailer_options(seg):
    opts = retail_opts(seg)
    return opts, opts[0]["value"] if opts else ([], None)

@app.callback(
    Output("brand-dd", "options"), Output("brand-dd", "value"),
    Input("retail-dd", "value")
)
def update_brand_options(ret):
    if not ret:
        return [], None
    opts = brand_opts(ret)
    return opts, opts[0]["value"]

@app.callback(
    Output("sales-fig", "figure"),
    Input("brand-dd", "value")
)
def update_sales_fig(brand):
    return build_sales_fig(brand)

@app.callback(
    Output("cap-bar", "figure"),
    Input("seg-dd", "value")
)
def update_capture_bar(seg):
    return capture_bar(seg)

@app.callback(
    Output("scatter-ac", "figure"),
    Input("seg-dd", "value")
)
def update_scatter(seg):
    return scatter_activity_capture(seg)

@app.callback(
    Output("insight-ul", "children"),
    Input("seg-dd", "value"), Input("retail-dd", "value"), Input("brand-dd", "value")
)
def update_insight_box(seg, ret, brand):
    lines = insights(seg, ret, brand)
    return [html.Li(l) for l in lines]

if __name__ == "__main__":
    app.run(host='127.0.0.1', port=8053, debug=True)
